Julia is a [Homoiconic](https://en.wikipedia.org/wiki/Homoiconicity) language. Meaning the code itself can be represented as a Julia data strcuture. We can parse a valid julia code as follows:

In [1]:
program = "1 + 2"
ex = Meta.parse(program)

:(1 + 2)

In [2]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 2


An expression has two parts, a `Symbol` indicating the type of the Expression and arguments for the expression. Which inturn can be symbols or expressions or literals. 

In [3]:
ex.head

:call

In [4]:
ex.args

3-element Array{Any,1}:
  :+
 1  
 2  

Now that we know the strucutre of an expression we can construct expressions:

In [5]:
ex  = Expr(:call, :+, 1, 1)

:(1 + 1)

and evaluate them...

In [6]:
eval(ex)

2

In [7]:
Meta.show_sexpr(ex)

(:call, :+, 1, 1)

We can use these features of julia to build a simple symbolic differentiation function. (for polynomials, to make them work for general functions we'll need a table of derivatives of functions like e^x, sin, cos etc.)

Since we have multiple dispatch we define 𝒹 function on different types of arguments separately. 

In [8]:
𝒹(x::Number, y::Symbol) = 0

function 𝒹(x::Symbol, y::Symbol) 
    if x == y
        return 1
    else
        return 0
    end
end;

Now in order to differentiate on algebraic operations, we define separate functions for +,-,/,* and then a function which calls the appropriate function depending upon the first argument of the expression. 

Add and subtract functions are simple enough. 

In [9]:
function 𝒹_add(ex::Expr, y::Symbol)
    n = length(ex.args)
    new_args = Array{Any}(nothing, n)
    new_args[1] = :+
    for i in 2:n
        new_args[i] = 𝒹(ex.args[i], y)
    end
    return Expr(:call, new_args...)
end;

function 𝒹_subtract(ex::Expr, y::Symbol)
    n = length(ex.args)
    new_args = Array{Any}(nothing, n)
    new_args[1] = :-
    for i in 2:n
        new_args[i] = 𝒹(ex.args[i], y)
    end
    return Expr(:call, new_args...)
end;

In [10]:
ex = Meta.parse("x + x + y")
𝒹_add(ex, :x)

:(1 + 1 + 0)

For multiply we need the product rule,

$ d(f*g) = d(f)*g + f*d(g) $

But since `*` can have many arguments, 

$ d(f*g*h) = d(f)*g*h + f*d(g)*h + f*g*d(h) $

If there are `n` arguments then there are `n` multiplications all need to be combined by a `+` call expression.
We can implement each of the `*` expressions by traversing the arguments to the expression and each time we differentiate one argument while keeping the others same.

In [11]:
function 𝒹_multiply(ex::Expr, y::Symbol)
    n = length(ex.args)
    sum_args = Array{Any}(nothing, n)
    sum_args[1] = :+
    for i in 2:n
        prod_args = Array{Any}(nothing, n)
        prod_args[1] = :*
        for j in 2:n
            if j == i
                prod_args[j] = 𝒹(ex.args[j], y)
            else
                prod_args[j] = ex.args[j]
            end
        end
        sum_args[i] = Expr(:call, prod_args)
    end
    return Expr(:call, sum_args)
end

                

𝒹_multiply (generic function with 1 method)

In [12]:
ex = Meta.parse("2*x")
𝒹_multiply(ex, :x)

:((Any[:+, :((Any[:*, 0, :x])()), :((Any[:*, 2, 1])())])())

For divide we do not need loops as it accepts two arguments we can manually construct expressions for the division rule

$ d(f/g) = d(f)/g - f*d(g)/g*g $

In [13]:
function 𝒹_divide(ex::Expr, y::Symbol)
    neg_args = Array{Any}(nothing, 3)
    neg_args[1] = :-
    
    neg_args[2] = Expr(:call, :/, 𝒹(ex.args[2], y), ex.args[3])
    
    neg_args[3] = Expr(:call, :/, (Expr(:call, :*, ex.args[2], 𝒹(ex.args[3], y))), Expr(:call, :*, ex.args[3], ex.args[3]))
    
    return Expr(:call, neg_args)
end


𝒹_divide (generic function with 1 method)

In [14]:
ex = Meta.parse("y/x")
𝒹_divide(ex, :x)

:((Any[:-, :(0 / x), :((y * 1) / (x * x))])())

In [15]:
function 𝒹(ex::Expr, y::Symbol)
    if ex.head == :call
        if ex.args[1] == :+
            return 𝒹_add(ex, y)
        elseif ex.args[1] == :-
            return 𝒹_minus(ex, y)
        elseif ex.args[1] == :*
            return 𝒹_multiply(ex, y)
        elseif ex.args[1] == :/
            return 𝒹_divide(ex, y)
        end
    else
        return 𝒹(ex.head)
    end
end
        

𝒹 (generic function with 3 methods)

In [16]:
𝒹(:(x + x*x), :x)

:(1 + (Any[:+, :((Any[:*, 1, :x])()), :((Any[:*, :x, 1])())])())